## ランダムフォレスト　デフォルト

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import tree


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55323, 15)

In [4]:
train_pkl.describe()

,id,accommodates,bathrooms,bedrooms,beds,host_response_rate,number_of_reviews,y,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Private room,room_type_Shared room,property_type_num
count,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,5.532300e+04,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000
mean,27787.181588,0.087360,0.248414,0.111274,0.238273,2.311836e-17,0.204165,160.307341,0.257253,0.437467,0.001356,0.000235,0.413806,0.028975,0.999259
std,16043.404061,0.790027,0.782466,0.794160,0.712608,1.000009e+00,0.752675,168.266655,0.437124,0.496079,0.036795,0.015328,0.492519,0.167739,0.075455
min,0.000000,-1.000000,-2.000000,-1.000000,-1.000000,-1.651052e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13892.500000,0.000000,0.000000,0.000000,0.000000,-1.651052e+00,0.000000,74.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,27778.000000,0.000000,0.000000,0.000000,0.000000,6.706177e-01,0.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,41681.500000,0.000000,0.000000,0.000000,0.000000,6.706177e-01,0.000000,185.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,55582.000000,5.000000,11.000000,10.000000,12.000000,6.706177e-01,15.000000,1999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


## 訓練データとテストデータに分割

In [5]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [6]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

## 訓練データで訓練

In [7]:
RF = RandomForestRegressor(random_state=42)

In [8]:
RF = RF.fit(train_X, train_y)

In [9]:
RF.feature_importances_

array([0.08321777, 0.20059379, 0.37212988, 0.05659658, 0.08299639,
       0.00909627, 0.02069459, 0.02326199, 0.00050718, 0.0009484 ,
       0.11473622, 0.03049206, 0.00472888])

In [10]:
sorted(
    zip(map(lambda x: round(x, 3), RF.feature_importances_), train.iloc[:, [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]].columns),
    reverse=True)

[(0.372, 'beds'),
 (0.201, 'bedrooms'),
 (0.115, 'room_type_Shared room'),
 (0.083, 'number_of_reviews'),
 (0.083, 'bathrooms'),
 (0.057, 'host_response_rate'),
 (0.03, 'property_type_num'),
 (0.023, 'cancellation_policy_super_strict_30'),
 (0.021, 'cancellation_policy_strict'),
 (0.009, 'cancellation_policy_moderate'),
 (0.001, 'room_type_Private room'),
 (0.001, 'cancellation_policy_super_strict_60')]

In [11]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(RF, out_file=None,
#                                feature_names=train.iloc[:, 1:].columns,
#                                class_names=train.Survived.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

## テストデータで実行

In [12]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [13]:
test_x.shape, test_y.shape

((11065, 13), (11065,))

In [14]:
pred_y = RF.predict(test_x)

In [15]:
np.sqrt(mean_squared_error(test_y, pred_y))

131.94898070968895

## 検証データで実行

In [16]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [17]:
valid.shape

(18528, 14)

In [18]:
# ID の保存
valid_pass = valid.id.values

In [19]:
valid_X = valid.iloc[:, 1:].values

In [20]:
# valid_X.describe()

In [21]:
valid_X.shape, train_X.shape

((18528, 13), (44258, 13))

In [22]:
pred_valid_y = RF.predict(valid_X)

In [23]:
pred_valid_y.shape

(18528,)

In [24]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [25]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [26]:
result_df.head()

,y
0,265.721349
1,153.785442
2,106.462799
3,189.421272
4,167.948985


In [27]:
result_df.to_csv("./RF_8.csv", header=False)